In [1]:
%cd ../
%cd weights

c:\Users\perez\OneDrive\Documents\GitHub\Yolo
c:\Users\perez\OneDrive\Documents\GitHub\Yolo\weights


In [2]:
import cv2

In [3]:
classes = [] # Gets our Object names
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:
import numpy as np
import time

In [5]:
cap=cv2.VideoCapture(0)

In [6]:
def predict(image, net):
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (64, 64), swapRB=True, crop=False)
    net.setInput(blob)
    preds = net.forward()
    return preds

In [7]:
def model(path):
    net = cv2.dnn.readNetFromONNX(path)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
    net.setPreferableTarget(cv2.dnn.DNN_BACKEND_DEFAULT)
    
    return net
    

In [8]:
def format_yolov5(frame):

    row, col, _ = frame.shape
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = frame
    return result

In [9]:
SCORE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.4
CONFIDENCE_THRESHOLD = 0.4

In [10]:
def wrap_detection(input_image, output_data):
    class_ids = []
    confidences = []
    boxes = []

    rows = output_data.shape[0]

    image_width, image_height, _ = input_image.shape

    x_factor = image_width / 64
    y_factor =  image_height / 64

    for r in range(rows):
        row = output_data[r]
        confidence = row[4]
        if confidence >= 0.4:

            classes_scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(classes_scores)
            class_id = max_indx[1]
            if (classes_scores[class_id] > .25):

                confidences.append(confidence)

                class_ids.append(class_id)

                x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.45) 

    result_class_ids = []
    result_confidences = []
    result_boxes = []

    for i in indexes:
        result_confidences.append(confidences[i])
        result_class_ids.append(class_ids[i])
        result_boxes.append(boxes[i])

    return result_class_ids, result_confidences, result_boxes

In [11]:
import cv2
import time
import sys



colors = [(255, 255, 0), (0, 255, 0), (0, 255, 255), (255, 0, 0)]

net = model("yolov5_half_export.onnx")

start = time.time_ns()
frame_count = 0
total_frames = 0
fps = -1

while True:

    _, frame = cap.read()
    if frame is None:
        print("End of stream")
        break

    inputImage = format_yolov5(frame)
    outs = predict(inputImage, net)

    class_ids, confidences, boxes = wrap_detection(inputImage, outs[0])

    frame_count += 1
    total_frames += 1

    for (classid, confidence, box) in zip(class_ids, confidences, boxes):
         color = colors[int(classid) % len(colors)]
         cv2.rectangle(frame, box, color, 2)
         cv2.rectangle(frame, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)
         cv2.putText(frame, classes[classid], (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,0))

    if frame_count >= 30:
        end = time.time_ns()
        fps = 1000000000 * frame_count / (end - start)
        frame_count = 0
        start = time.time_ns()
    
    if fps > 0:
        fps_label = "FPS: %.2f" % fps
        cv2.putText(frame, fps_label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("output", frame)
    print("Total frames: " + str(total_frames))
    if cv2.waitKey(1) > -1:
        print("finished by user")
        break


Total frames: 1
Total frames: 2
Total frames: 3
Total frames: 4
Total frames: 5
Total frames: 6
Total frames: 7
Total frames: 8
Total frames: 9
Total frames: 10
Total frames: 11
Total frames: 12
Total frames: 13
Total frames: 14
Total frames: 15
Total frames: 16
Total frames: 17
Total frames: 18
Total frames: 19
Total frames: 20
Total frames: 21
Total frames: 22
Total frames: 23
Total frames: 24
Total frames: 25
Total frames: 26
Total frames: 27
Total frames: 28
Total frames: 29
Total frames: 30
Total frames: 31
Total frames: 32
Total frames: 33
Total frames: 34
Total frames: 35
Total frames: 36
Total frames: 37
Total frames: 38
Total frames: 39
Total frames: 40
Total frames: 41
Total frames: 42
Total frames: 43
Total frames: 44
Total frames: 45
Total frames: 46
Total frames: 47
Total frames: 48
Total frames: 49
Total frames: 50
Total frames: 51
Total frames: 52
Total frames: 53
Total frames: 54
Total frames: 55
Total frames: 56
Total frames: 57
Total frames: 58
Total frames: 59
Total 

KeyboardInterrupt: 

In [12]:
cap.release()
cv2.destroyAllWindows()